In [23]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from helpers import preprocessing_pipeline, count_syntactic_features

In [24]:
raw_train_set = pd.read_csv("data/train.csv")
train_set = preprocessing_pipeline(raw_train_set)


In [25]:
new_columns = pd.DataFrame(columns=["neg", "neu", "pos", "compound", 'Stopwords', 'Nouns', 'Verbs', 'Adverbs', 'Adjectives', 'Pronouns', "length"])
train_set = train_set.join(new_columns)
train_set.head()


,tweets,class,neg,neu,pos,compound,Stopwords,Nouns,Verbs,Adverbs,Adjectives,Pronouns,length
21238,fav moment in sepp blatter vid ( 0:20 ) : `` w...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21239,just found this while walking my human ....,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21240,'disrespected the wife of prophet ' - pseudo l...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21241,do you know that super yeay satisfying feeling...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21242,if you 're going to call someone ignorant and ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for index, row in train_set.iterrows():
    tweet = row[0]
    scores = SentimentIntensityAnalyzer().polarity_scores(tweet)
    
    for sentiment, score in scores.items():
        train_set.loc[index, sentiment] = score
        
    syntax_counts = count_syntactic_features(tweet)
    for syntax, count in syntax_counts.items():
        train_set.loc[index, syntax] = count
        
    train_set.loc[index, "length"] = len(tweet)
    
train_set = train_set.drop_duplicates()
train_set = train_set.reset_index(drop=True)
train_set.head()

,tweets,class,neg,neu,pos,compound,Stopwords,Nouns,Verbs,Adverbs,Adjectives,Pronouns,length
21238,fav moment in sepp blatter vid ( 0:20 ) : `` w...,1,0.000,0.778,0.222,0.6908,10.0,5.0,1.0,1.0,2.0,0.0,116.0
21239,just found this while walking my human ....,1,0.000,1.000,0.000,0.0000,4.0,2.0,2.0,0.0,0.0,0.0,43.0
21240,'disrespected the wife of prophet ' - pseudo l...,1,0.217,0.652,0.130,-0.2960,3.0,6.0,2.0,0.0,0.0,0.0,80.0
21241,do you know that super yeay satisfying feeling...,1,0.000,0.704,0.296,0.8126,11.0,3.0,6.0,1.0,1.0,0.0,120.0
21242,if you 're going to call someone ignorant and ...,1,0.234,0.766,0.000,-0.6705,9.0,3.0,4.0,1.0,3.0,0.0,104.0


In [5]:
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(train_set["tweets"])
bow_data = pd.DataFrame(bag_of_words.toarray(), columns=vectorizer.get_feature_names())
bow_data.head()

/Users/haim/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,000009,000ft,000km,000th,000yrs,001,003,007,...,आरक,छड,नव,वर,षण,सरक,ભક,રક,ષક,ᵀºᵗªˡˡʸ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y = train_set["class"]
train_set.drop(columns=["tweets", "class"], inplace=True)

In [19]:
X = train_set[:1].join(bow_data[:1])
X.head()

ValueError: columns overlap but no suffix specified: Index(['pos', 'compound', 'length'], dtype='object')

In [17]:
train_set = train_set.reset_index(drop=True)

,neg,neu,pos,compound,Stopwords,Nouns,Verbs,Adverbs,Adjectives,Pronouns,length
